In [1]:
import sys
import os
sys.path.append('..')
from lib.path import get_training_data_dir, get_predicted_labeled_water_path, get_displaceable_water_path, get_non_displaceable_water_path
from lib.pdb import filter_atoms_and_create_new_pdb
from data_loader.DataLoaderForTest import DataLoaderForTest
from models.LeNet import LeNet
from lib.helper import get_latest_checkpoint
from tensorflow.keras.losses import BinaryCrossentropy
from custom_losses.dice import dice_loss, dice_coefficient
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

2024-06-08 08:51:13.478595: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 08:51:13.481103: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-08 08:51:13.513032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 08:51:13.513052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 08:51:13.514245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
DATA_TYPE = 'gr'
DATA_VOXEL_NUM = 10
CLASSIFYING_RULE = 'WaterClassifyingRuleSurface'
LIGAND_POCKET_DEFINER = 'LigandPocketDefinerOriginal'
LIGAND_VOXEL_NUM = 8
MODEL_NAME = 'LeNet'
test_dir = '/mnt/ito/test/WD5/test_data/'
data_loader = DataLoaderForTest(test_dir)
test_data, test_water_ids = data_loader.get_test_data_and_water_ids('3smrA')

In [3]:
test_data.shape

(28, 21, 21, 21, 1)

In [ ]:
model_func = ResNet
MODEL_NAME = model_func.__name__
input_shape = (DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, test_data_displaceable.shape[-1])
n_base = 8
learning_rate = 1e-5
metrics = ['accuracy', dice_coefficient, Recall(), Precision()]
BN = True

# Load the model and its weights
checkpoint_dir = f'../checkpoints/{DATA_TYPE}/data_voxel_num_{DATA_VOXEL_NUM}/{LIGAND_POCKET_DEFINER}/ligand_pocket_voxel_num_{LIGAND_VOXEL_NUM}/{CLASSIFYING_RULE}/{MODEL_NAME}/aug_train/'
latest_checkpoint = get_latest_checkpoint(checkpoint_dir)
model = model_func(n_base, input_shape, learning_rate, BinaryCrossentropy(), metrics, BN=BN)
model.load_weights(latest_checkpoint)

# Predict the labels
prediction = model.predict(test_data)
prediction_values = prediction.reshape(prediction.shape[0])
optimal_threshold_youden = calculate_optimal_threshold(test_data_label, prediction_values)
print(optimal_threshold_youden)
prediction_displaceable = model.predict(test_data_displaceable)
predicted_labels_dis = custom_threshold(prediction_displaceable, optimal_threshold_youden)

prediction_non_displaceable = model.predict(test_data_non_displaceable)
predicted_labels_non_dis = custom_threshold(prediction_non_displaceable, optimal_threshold_youden)



def extract_ids(labels, ids_array, condition):
    indices = np.where(labels == condition)[0]  # np.where returns indices that match the condition
    return ids_array[indices]

# Extract IDs based on predictions for all conditions
water_ids_TP = extract_ids(predicted_labels_dis, dis_water_ids, 1)
water_ids_FN = extract_ids(predicted_labels_dis, dis_water_ids, 0)
water_ids_FP = extract_ids(predicted_labels_non_dis, non_dis_water_ids, 1)
water_ids_TN = extract_ids(predicted_labels_non_dis, non_dis_water_ids, 0)

# Save and process each type of data
labels = ["TP", "FN", "FP", "TN"]
ids_list = [water_ids_TP, water_ids_FN, water_ids_FP, water_ids_TN]
print(ids_list)
for label, water_ids in zip(labels[:2], ids_list[:2]):
    output_path = get_predicted_labeled_water_path(DATA_TYPE, pdb_name, DATA_VOXEL_NUM, LIGAND_VOXEL_NUM,
                                                   CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, MODEL_NAME, label, is_augmented)
    input_path = get_displaceable_water_path(pdb_name, LIGAND_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER)
    
    # Process the predicted data to filter and create a new PDB for each label type
    filter_atoms_and_create_new_pdb(input_path, output_path, water_ids)

for label, water_ids in zip(labels[2:], ids_list[2:]):
    output_path = get_predicted_labeled_water_path(DATA_TYPE, pdb_name, DATA_VOXEL_NUM, LIGAND_VOXEL_NUM,
                                                   CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, MODEL_NAME, label, is_augmented)
    input_path = get_non_displaceable_water_path(pdb_name, LIGAND_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER)
    
    # Process the predicted data to filter and create a new PDB for each label type
    filter_atoms_and_create_new_pdb(input_path, output_path, water_ids)


In [5]:
input_shape = (DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, 1)
n_base = 16
learning_rate = 1e-5
metrics = ['accuracy', dice_coefficient, Recall(), Precision()]
BN = True

2024-04-22 09:18:54.950659: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
checkpoint_dir = f'../checkpoints/{DATA_TYPE}/data_voxel_num_{DATA_VOXEL_NUM}/{LIGAND_POCKET_DEFINER}/ligand_pocket_voxel_num_{LIGAND_VOXEL_NUM}/{CLASSIFYING_RULE}/{MODEL_NAME}/'
latest_checkpoint = get_latest_checkpoint(checkpoint_dir)
model = LeNet(n_base, input_shape, learning_rate, BinaryCrossentropy(), metrics, BN=BN)
model.load_weights(latest_checkpoint)


/home/ito/anaconda3/envs/yuki_research/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 21, 21, 21, 16) │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 21, 21, 21, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 10, 10, 10, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 10, 10, 10, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 10, 10, 10, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 5, 5, 5, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       128,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,561 (556.88 KB)

 Trainable params: 142,465 (556.50 KB)

 Non-trainable params: 96 (384.00 B)

/home/ito/anaconda3/envs/yuki_research/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [10]:
prediction_displaceable = model.predict(test_data_displaceable)
prediction_values_displaceable = prediction_displaceable.reshape(prediction_displaceable.shape[0])
predicted_label_dis = prediction_displaceable.round()

prediction_non_displaceable = model.predict(test_data_non_displaceable)
prediction_values_non_displaceable = prediction_non_displaceable.reshape(prediction_non_displaceable.shape[0])
predicted_label_non_dis = prediction_non_displaceable.round()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


In [11]:
indices_of_TP = np.where(predicted_label_dis == 1)[0] # np.where returns 2D indices
indices_of_FN = np.where(predicted_label_dis == 0)[0]

indices_of_FP = np.where(predicted_label_non_dis == 1)[0] # np.where returns 2D indices
indices_of_TN = np.where(predicted_label_non_dis == 0)[0]

In [12]:
water_ids_TP = dis_water_ids[indices_of_TP]
water_ids_FN = dis_water_ids[indices_of_FN]
water_ids_FP = non_dis_water_ids[indices_of_FP]
water_ids_TN = non_dis_water_ids[indices_of_TN]

In [13]:
prediction_values_TP = prediction_values_displaceable[indices_of_TP]
prediction_values_FN = prediction_values_displaceable[indices_of_FN]
prediction_values_FP = prediction_values_non_displaceable[indices_of_FP]
prediction_values_TN = prediction_values_non_displaceable[indices_of_TN]

In [17]:
water_id2prediction_value_TP = {water_id : prediction_value for water_id, prediction_value in zip(water_ids_TP, prediction_values_TP)}
water_id2prediction_value_FN = {water_id : prediction_value for water_id, prediction_value in zip(water_ids_FN, prediction_values_FN)}

water_id2prediction_value_FP = {water_id : prediction_value for water_id, prediction_value in zip(water_ids_FP, prediction_values_FP)}
water_id2prediction_value_TN = {water_id : prediction_value for water_id, prediction_value in zip(water_ids_TN, prediction_values_TN)}

print("TP: ", water_id2prediction_value_TP)
print("FN: ", water_id2prediction_value_FN)
print("FP: ", water_id2prediction_value_FP)
print("TN: ", water_id2prediction_value_TN)

TP:  {941: 0.6366372, 666: 0.7736607, 1069: 0.6957607, 216: 0.80631757, 667: 0.82130253, 131: 0.5225806}
FN:  {656: 0.30708146, 420: 0.13442233, 711: 0.47150803, 129: 0.29293793, 788: 0.17954832, 561: 0.34224707, 188: 0.24387303, 229: 0.27798364, 34: 0.2891343}
FP:  {550: 0.6481192, 671: 0.5033118, 668: 0.5660274}
TN:  {682: 0.30184636, 708: 0.25542265, 491: 0.24988605, 458: 0.3332024, 81: 0.15289299, 1028: 0.32673353, 559: 0.4017425, 881: 0.32944325, 78: 0.3838709, 95: 0.28970626, 706: 0.46545634, 372: 0.36406153, 48: 0.17018513}


In [15]:
output_path = get_predicted_labeled_water_path(DATA_TYPE,
                                 '4lkk',
                                  DATA_VOXEL_NUM,
                                  LIGAND_VOXEL_NUM,
                                  CLASSIFYING_RULE,
                                  LIGAND_POCKET_DEFINER,
                                  MODEL_NAME,
                                  "TP")
input_path = get_displaceable_water_path('4lkk', LIGAND_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER)